In [9]:
#Dependencies
import pandas as pd
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import gmaps
from api_key import apikey 
from api_key import g_key 
from pytrends.request import TrendReq
import time

In [2]:
gmaps.configure(api_key=g_key)

In [3]:
#Set Search URL
search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id=259&entity_type=city"

#Create Requests
print(requests.get(search_url, headers={"user-key" : apikey}))

#Get Data and Convert into JSON
data = requests.get(search_url, headers={"user-key" : apikey}).json()

#Display JSON Data
data

<Response [200]>


{'results_found': 17502,
 'results_start': 0,
 'results_shown': 20,
 'restaurants': [{'restaurant': {'R': {'has_menu_status': {'delivery': -1,
      'takeaway': -1},
     'res_id': 16577492,
     'is_grocery_store': False},
    'apikey': 'c4ae10a41c1f43f16338d7cc86e554cc',
    'id': '16577492',
    'name': 'Chin Chin',
    'url': 'https://www.zomato.com/melbourne/chin-chin-3-cbd?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1',
    'location': {'address': '125 Flinders Lane, CBD, Melbourne',
     'locality': 'CBD',
     'city': 'Melbourne',
     'city_id': 259,
     'latitude': '-37.8155645773',
     'longitude': '144.9703757465',
     'zipcode': '3000',
     'country_id': 14,
     'locality_verbose': 'CBD, Melbourne'},
    'switch_to_order_menu': 0,
    'cuisines': 'Thai, Asian Fusion',
    'timings': '11 AM to 11 PM (Mon-Thu, Sun), 11 AM to 11:30 PM (Fri-Sat)',
    'average_cost_for_two': 135,
    'price_range': 4,
    'currency': '$',
    'highlights': ['Dinner',
     'Cr

In [4]:
#Find Number of Restaurants
resto_num = data["results_found"]

#Print Number of Restaurants
print(f"The number of restaurants in Melbourne, Sydney and Perth is {resto_num}")

The number of restaurants in Melbourne, Sydney and Perth is 17502


In [5]:
#Set City IDs
cityid = [259,260,296,2555,313,2171,2246,2298,1701,8253]

#Create Empty Lists
rest_id_list = []
rest_name_list =[]
rest_cuis_list = []
rest_local_list = []
rest_city_list = []
rest_lat_list = []
rest_long_list = []
rest_rate_list = []
rest_delivery_list = []
rest_ratenum_list = []

resto_num = 20
loop_num = 0
show_page = 0

#Create For Loop
for citynum in cityid:
    search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id={citynum}&entity_type=city&start={show_page}"
    data = requests.get(search_url, headers={"user-key" : apikey}).json()
    for page in range(5):
        for count in range(resto_num):
            try: 
                rest_id = data["restaurants"][count]["restaurant"]["id"]
                rest_name = data["restaurants"][count]["restaurant"]["name"]
                rest_cuis = data["restaurants"][count]["restaurant"]["cuisines"]
                rest_local = data["restaurants"][count]["restaurant"]["location"]["locality"]
                rest_city = data["restaurants"][count]["restaurant"]["location"]["city"]
                rest_lat = data["restaurants"][count]["restaurant"]["location"]["latitude"]
                rest_long = data["restaurants"][count]["restaurant"]["location"]["longitude"]
                rest_rate_num = data["restaurants"][count]["restaurant"]["all_reviews_count"]
                rest_rate = data["restaurants"][count]["restaurant"]["user_rating"]["aggregate_rating"]
                rest_delivery = data["restaurants"][count]["restaurant"]["highlights"]
                
                #Check if they Deliver 
                theydelivery = False
                for doesdelivery in rest_delivery:
                    if doesdelivery == 'Delivery':
                        theydelivery = True

                if theydelivery == True:
                    rest_delivery = "Yes"
                else:
                    rest_delivery = "No"

                #Append Results to List
                rest_id_list.append(rest_id)
                rest_name_list.append(rest_name)
                rest_cuis_list.append(rest_cuis)
                rest_local_list.append(rest_local)
                rest_city_list.append(rest_city)
                rest_lat_list.append(rest_lat)
                rest_long_list.append(rest_long)
                rest_rate_list.append(rest_rate)
                rest_ratenum_list.append(rest_rate_num)
                rest_delivery_list.append(rest_delivery)
                
                #Repeat
                if count == 19:
                    show_page = show_page + 20
                    search_url = f"https://developers.zomato.com/api/v2.1/search?entity_id={citynum}&entity_type=city&start={show_page}"
                    data = requests.get(search_url, headers={"user-key" : apikey}).json()
                    count = 0
            except:
                print("error")
    count = 0
    page = 0
    show_page = 0


#Check for Errors
try:            
    restaurants_df = pd.DataFrame({
        "Restaurant ID":rest_id_list,
        "Name":rest_name_list,
        "Cuisine":rest_cuis_list,
        "Locality":rest_local_list,
        "City":rest_city_list,
        "Latitude":rest_lat_list,
        "Longitude":rest_long_list,
        "Rating":rest_rate_list,
        "Total Number Ratings": rest_ratenum_list,
        "Delivery":rest_delivery_list
    })
except:
    print(len(rest_id_list))
    print(len(rest_name_list))
    print(len(rest_cuis_list))
    print(len(rest_local_list))
    print(len(rest_city_list))
    print(len(rest_lat_list))
    print(len(rest_long_list))
    print(len(rest_rate_list))
    print(len(rest_review_list))
    print(len(rest_delivery_list))
    
#Drop Duplicate Results    
restaurants_df = restaurants_df.drop_duplicates()

#Print DataFrame
restaurants_df

error
error
error
error
error
error
error
error


,Restaurant ID,Name,Cuisine,Locality,City,Latitude,Longitude,Rating,Total Number Ratings,Delivery
0,16577492,Chin Chin,"Thai, Asian Fusion",CBD,Melbourne,-37.8155645773,144.9703757465,4.6,3206,No
1,16585905,Tipo 00,Italian,CBD,Melbourne,-37.8135277429,144.9619733915,4.9,721,No
2,16577212,Beatrix,"Coffee and Tea, Bakery",North Melbourne,Melbourne,-37.8023673299,144.9444753304,4.9,384,No
3,16583098,Lune Croissanterie,"Bakery, French",Fitzroy,Melbourne,-37.7960258682,144.9799244106,4.9,753,No
4,16573665,The Hardware Societe,"French, Cafe Food, Coffee and Tea",CBD,Melbourne,-37.8120317331,144.9604676664,4.6,2103,No
...,...,...,...,...,...,...,...,...,...,...
987,17554136,Country Cafe,,Mangum,Mangum,34.8723000000,-99.4974000000,0,0,No
988,17553277,Sunshine Cafe',"American, Pizza, Cafe",Cordell,Cordell,35.2914000000,-98.9900000000,0,0,Yes
989,18936306,Shorty Bauman's,Burger,Cordell,Cordell,35.2909430000,-98.9871340000,0,0,No
990,17553284,Sonic Drive-In,Fast Food,Cordell,Cordell,35.3038048209,-98.9879744538,0,0,No


In [6]:
#Save DataFrame to csv 
restaurants_df.to_csv("Multi_City_Data.csv", encoding="utf-8", index=False)

In [7]:
#Number of Restaurants that Deliver
restaurants_df["Delivery"].value_counts()

No     897
Yes     70
Name: Delivery, dtype: int64

In [8]:
#Sort DataFrame by Rating
restaurants_df = restaurants_df.sort_values(by=["Rating"],ascending=False)

#Print DataFrame
reshaped = \
(restaurants_df.set_index(restaurants_df.columns.drop('Cuisine',1).tolist())
   .Cuisine.str.split(', ', expand=True)
   .stack()
   .reset_index()
   .rename(columns={0:'Cuisine'})
   .loc[:, restaurants_df.columns]
)

reshaped_df = pd.DataFrame(reshaped)
reshaped_df

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
#Count Number of Cuisines
top_cuisines = reshaped_df["Cuisine"].value_counts()

#Get Top 10
top_10_cuisines = top_cuisines.iloc[[0,1,2,3,4,5,6,7,8,9]]

#Print Results
top_cuisines

In [ ]:
#Plot as Bar Graph
top_10_cuisines.plot(kind='bar')

In [ ]:
#Get Top Restaurants by City
perth_top = reshaped_df.loc[reshaped_df["City"]=="Perth"]

#Print DataFrame
perth_top.head(10)

In [ ]:
top_P_cuisines = perth_top["Cuisine"].value_counts()

#Get Top 10
top_10_P_cuisines = top_P_cuisines.iloc[[0,1,2,3,4,5,6,7,8,9]]

#Print Results
top_P_cuisines


top_10_P_cuisines.plot(kind='bar')

In [ ]:
#Get Top Restaurants by City
melbourne_top = reshaped_df.loc[reshaped_df["City"]=="Melbourne"]

#Print DataFrame
melbourne_top.head(10)

In [ ]:
top_M_cuisines = melbourne_top["Cuisine"].value_counts()

#Get Top 10
top_10_M_cuisines = top_M_cuisines.iloc[[0,1,2,3,4,5,6,7,8,9]]

#Print Results
top_M_cuisines


top_10_M_cuisines.plot(kind='bar')

In [ ]:
#Get Top Restaurants by City
sydney_top = reshaped_df.loc[reshaped_df["City"]=="Sydney"]

#Print DataFrame
sydney_top.head(10)

In [ ]:
top_S_cuisines = sydney_top["Cuisine"].value_counts()

#Get Top 10
top_10_S_cuisines = top_S_cuisines.iloc[[0,1,2,3,4,5,6,7,8,9]]

#Print Results
top_S_cuisines


top_10_S_cuisines.plot(kind='bar')

In [ ]:
locations = melbourne_top[["Latitude", "Longitude"]].astype(float)
rating = melbourne_top["Rating"].astype(float)

figure = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                 dissipating=True, max_intensity=50, point_radius=20)
figure.add_layer(markers)
figure.add_layer(heat_layer)

figure

In [ ]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>Rating</dt><dd>{Rating}</dd>

</dl>
"""

Name_info = [info_box_template.format(**row) for index, row in melbourne_top.iterrows()]
locations = melbourne_top[["Latitude", "Longitude"]].astype(float)

markers = gmaps.marker_layer(locations, info_box_content=Name_info)

In [ ]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>Rating</dt><dd>{Rating}</dd>

</dl>
"""

Name_info2 = [info_box_template.format(**row) for index, row in sydney_top.iterrows()]
locations2 = sydney_top[["Latitude", "Longitude"]].astype(float)

markers2 = gmaps.marker_layer(locations, info_box_content=Name_info2)

In [ ]:
locations = sydney_top[["Latitude", "Longitude"]].astype(float)
rating = sydney_top["Rating"].astype(float)

figure = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                 dissipating=True, max_intensity=50, point_radius=10)
figure.add_layer(heat_layer)
figure.add_layer(markers)
figure

In [ ]:
locations = perth_top[["Latitude", "Longitude"]].astype(float)
rating = perth_top["Rating"].astype(float)

figure = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                 dissipating=True, max_intensity=50, point_radius=10)
figure.add_layer(heat_layer)

figure